# AutoGluon: Beginner Multimodal Quick Start

## Objective
This notebook demonstrates **multimodal learning** using AutoGluon, combining multiple data types (text, images, tabular) in a single model for improved predictions.

## Use Case
Multimodal learning is useful for:
- E-commerce product classification (images + descriptions + features)
- Social media content analysis (images + text + metadata)
- Healthcare diagnosis (medical images + patient records + notes)
- Real estate valuation (photos + descriptions + property features)
- Content recommendation (multiple content types)

## Key Concepts
- Combines different data modalities
- Automatic feature extraction from each modality
- Joint learning across modalities
- More information = better predictions

In [ ]:
# Install AutoGluon
!pip install -q autogluon

In [ ]:
# Import libraries
import pandas as pd
import numpy as np
from autogluon.tabular import TabularDataset, TabularPredictor
import os
import shutil

In [ ]:
# Load dataset
# TODO: Upload your multimodal dataset or use URL
# Dataset can include multiple data types:
# - Images: column with image file paths
# - Text: columns with text descriptions, reviews, etc.
# - Numerical: columns with numeric features
# - Categorical: columns with category labels
# - Label: target variable

# Example: train_data = TabularDataset('path/to/multimodal_data.csv')

train_data = None  # Replace with your data
test_data = None   # Replace with your data

print("Dataset loaded successfully!")
if train_data is not None:
    print(f"Training data shape: {train_data.shape}")
    print("\nColumn types:")
    print(train_data.dtypes)
    print("\nSample data:")
    print(train_data.head())

In [ ]:
# Set label column
LABEL = 'label'  # TODO: Replace with your label column name

In [ ]:
# Auto-detect problem type based on label
if train_data is not None and LABEL in train_data.columns:
    # Check if the label is numeric (regression) or categorical (classification)
    if pd.api.types.is_numeric_dtype(train_data[LABEL]):
        # Check if it's continuous or discrete
        unique_ratio = train_data[LABEL].nunique() / len(train_data)
        if unique_ratio > 0.05:  # More than 5% unique values suggests regression
            problem_type = 'regression'
            eval_metric = 'rmse'
        else:
            problem_type = 'classification'
            eval_metric = 'roc_auc'
    else:
        problem_type = 'classification'
        eval_metric = 'roc_auc'
else:
    # Default to classification
    problem_type = 'classification'
    eval_metric = 'roc_auc'

print(f"Problem Type: {problem_type}")
print(f"Evaluation Metric: {eval_metric}")
print("\nNote: AutoGluon automatically handles multiple data modalities.")

In [ ]:
# Train the model
# AutoGluon automatically detects and processes all data types
# No manual feature engineering needed!
predictor = TabularPredictor(
    label=LABEL,
    problem_type=problem_type,
    eval_metric=eval_metric,
    path='./autogluon-multimodal-model'
).fit(
    train_data=train_data,
    presets='medium_quality',
    time_limit=900
)

print("Model training completed!")
print("The model learned from all data modalities.")

In [ ]:
# Display and save leaderboard
leaderboard = predictor.leaderboard(test_data, silent=True)
print("\nModel Leaderboard:")
print(leaderboard)

# Save leaderboard to CSV
leaderboard.to_csv('leaderboard.csv', index=False)
print("\nLeaderboard saved to leaderboard.csv")

In [ ]:
# Display and save feature importance
try:
    feature_importance = predictor.feature_importance(test_data)
    print("\nFeature Importance:")
    print(feature_importance)
    print("\nNote: See which modalities contribute most to predictions.")
    
    # Save feature importance to CSV
    feature_importance.to_csv('feature_importance.csv')
    print("\nFeature importance saved to feature_importance.csv")
except Exception as e:
    print(f"Could not compute feature importance: {e}")

In [ ]:
# Make predictions
if test_data is not None:
    predictions = predictor.predict(test_data)
    print("\nPredictions:")
    print(predictions.head())
    
    # For classification, also show prediction probabilities
    if problem_type == 'classification':
        pred_probs = predictor.predict_proba(test_data)
        print("\nPrediction Probabilities:")
        print(pred_probs.head())
        
    # Example usage
    print("\nMultimodal prediction example:")
    print("new_data = pd.DataFrame({")
    print("    'image': ['path/to/image.jpg'],")
    print("    'text': ['Product description...'],")
    print("    'price': [49.99],")
    print("    'category': ['Electronics']")
    print("})")
    print("prediction = predictor.predict(new_data)")

In [ ]:
# Save model artifacts as zip file
model_path = './autogluon-multimodal-model'
zip_filename = 'autogluon_multimodal_model'

if os.path.exists(model_path):
    shutil.make_archive(zip_filename, 'zip', model_path)
    print(f"\nModel artifacts saved to {zip_filename}.zip")
else:
    print("Model path not found. Train the model first.")